## PyTorch Tutorial

IFT6135 – Representation Learning

A Deep Learning Course, January 2019

By Chin-Wei Huang 

(Adapted from Sandeep Subramanian's MILA tutorial)

### Torch Autograd, Define-by-run & Execution Paradigm

Adapted from
1. http://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html#sphx-glr-beginner-blitz-autograd-tutorial-py 
2. http://pytorch.org/docs/master/notes/autograd.html

## Variables : Thin wrappers around tensors to facilitate autograd

Supports almost all operations that can be performed on regular tensors

In [1]:
import numpy as np
from __future__ import print_function

In [2]:
import torch 

### Properties of Tensor : Requiring gradients, Data & Grad

1. You can access the raw tensor through the .data attribute
2. Gradient of the loss w.r.t. this variable is accumulated into .grad.
3. Stay tuned for requires_grad

In [3]:
z = torch.Tensor(5, 3).uniform_(-1, 1)

print(z.data)
print('Gradient : %s ' % (z.grad))
print('Requires Gradient : %s ' % (z.requires_grad))

tensor([[ 0.6914, -0.6770,  0.5039],
        [-0.6465, -0.0090,  0.5568],
        [-0.9361,  0.4825, -0.2124],
        [ 0.1490, -0.5000,  0.0219],
        [-0.3132, -0.4672,  0.5908]])
Gradient : None 
Requires Gradient : False 


## Define-by-run Paradigm

The torch autograd package provides automatic differentiation for all operations on Tensors.

PyTorch's autograd is a reverse mode automatic differentiation system.

Backprop is defined by how your code is run, and that every single iteration can be different.

Other frameworks that adopt a similar approach :

1. Chainer - https://github.com/chainer/chainer
2. DyNet - https://github.com/clab/dynet
3. Tensorflow Eager - https://research.googleblog.com/2017/10/eager-execution-imperative-define-by.html

### How autograd encodes execution history


Conceptually, autograd maintains a graph that records all of the operations performed on variables as you execute your operations. This results in a directed acyclic graph whose leaves are the input variables and roots are the output variables. By tracing this graph from roots to leaves, you can automatically compute the gradients using the chain rule.

![caption](images/dynamic_graph.gif)

GIF source: https://github.com/pytorch/pytorch

Internally, autograd represents this graph as a graph of Function objects (really expressions), which can be `apply()` ed to compute the result of evaluating the graph. When computing the forward pass, autograd simultaneously performs the requested computations and builds up a graph representing the function that computes the gradient (the `.grad_fn` attribute of each Variable is an entry point into this graph). When the forward pass is completed, we evaluate this graph in the backwards pass to compute the gradients.

In [4]:
x = torch.Tensor(5, 3).uniform_(-1, 1)
y = torch.Tensor(3, 5).uniform_(-1, 1)
z = torch.mm(x, y)
print(z.grad_fn)

None


An important thing to note is that the graph is recreated from scratch at every iteration, and this is exactly what allows for using arbitrary Python control flow statements, that can change the overall shape and size of the graph at every iteration. You don’t have to encode all possible paths before you launch the training - what you run is what you differentiate.

## Getting gradients : `backward()` & `torch.autograd.grad`

In [6]:
x = torch.Tensor(5, 3).uniform_(-1, 1).requires_grad_()
y= torch.Tensor(5, 3).uniform_(-1, 1).requires_grad_()
z = x ** 2 + 3 * y
print(z.grad_fn)

In [7]:
z.backward(gradient=torch.ones(5, 3))

In [8]:
# eq computes element-wise equality
torch.eq(x.grad, 2 * x)

tensor([[ 1,  1,  1],
        [ 1,  1,  1],
        [ 1,  1,  1],
        [ 1,  1,  1],
        [ 1,  1,  1]], dtype=torch.uint8)

In [12]:
y.grad

Variable containing:
 3  3  3
 3  3  3
 3  3  3
 3  3  3
 3  3  3
[torch.FloatTensor of size 5x3]

In [11]:
x = torch.Tensor(5, 3).uniform_(-1, 1).requires_grad_()
y = torch.Tensor(5, 3).uniform_(-1, 1).requires_grad_()
z = x ** 2 + 3 * y
dz_dx = torch.autograd.grad(z, x, grad_outputs=torch.ones(5, 3))
dz_dy = torch.autograd.grad(z, y, grad_outputs=torch.ones(5, 3))

## Define-by-run example

### Common Variable definition

In [12]:
x = torch.Tensor(5, 3).uniform_(-1, 1).requires_grad_()
w = torch.Tensor(3, 10).uniform_(-1, 1).requires_grad_()
b = torch.Tensor(10,).uniform_(-1, 1).requires_grad_()

### Graph 1 : `wx + b`

In [13]:
o = torch.matmul(x, w) + b
do_dinputs_1 = torch.autograd.grad(o, [x, w, b], grad_outputs=torch.ones(5, 10))

In [14]:
print('Gradients of o w.r.t inputs in Graph 1')
print('do/dx : \n\n %s ' % (do_dinputs_1[0]))
print('do/dw : \n\n %s ' % (do_dinputs_1[1]))
print('do/db : \n\n %s ' % (do_dinputs_1[2]))

Gradients of o w.r.t inputs in Graph 1
do/dx : 

 tensor([[ 0.2046, -0.5623,  3.0726],
        [ 0.2046, -0.5623,  3.0726],
        [ 0.2046, -0.5623,  3.0726],
        [ 0.2046, -0.5623,  3.0726],
        [ 0.2046, -0.5623,  3.0726]]) 
do/dw : 

 tensor([[-1.1262, -1.1262, -1.1262, -1.1262, -1.1262, -1.1262, -1.1262,
         -1.1262, -1.1262, -1.1262],
        [ 0.1220,  0.1220,  0.1220,  0.1220,  0.1220,  0.1220,  0.1220,
          0.1220,  0.1220,  0.1220],
        [-1.5374, -1.5374, -1.5374, -1.5374, -1.5374, -1.5374, -1.5374,
         -1.5374, -1.5374, -1.5374]]) 
do/db : 

 tensor([ 5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.]) 


### Graph 2 : wx / b

In [15]:
o = torch.matmul(x, w) / b
do_dinputs_2 = torch.autograd.grad(o, [x, w, b], grad_outputs=torch.ones(5, 10))

In [16]:
print('Gradients of o w.r.t inputs in Graph 2')
print('do/dx : \n %s ' % (do_dinputs_2[0]), (w/b[None,:]).sum(1))
print('do/dw : \n %s ' % (do_dinputs_2[1]), (x.sum(0)[:,None]/b[None,:]))
print('do/db : \n %s ' % (do_dinputs_2[2]))

Gradients of o w.r.t inputs in Graph 2
do/dx : 
 tensor([[ 23.8792,  27.4176,  20.7411],
        [ 23.8792,  27.4176,  20.7411],
        [ 23.8792,  27.4176,  20.7411],
        [ 23.8792,  27.4176,  20.7411],
        [ 23.8792,  27.4176,  20.7411]])  tensor([ 23.8792,  27.4176,  20.7411])
do/dw : 
 tensor([[ -5.2811, -24.8777,   1.2732,   3.2276,  -1.1894,  -3.3144,
           1.9645,   3.7445,   2.2350, -19.3046],
        [  0.5722,   2.6956,  -0.1380,  -0.3497,   0.1289,   0.3591,
          -0.2129,  -0.4057,  -0.2422,   2.0917],
        [ -7.2092, -33.9605,   1.7380,   4.4060,  -1.6236,  -4.5245,
           2.6817,   5.1116,   3.0511, -26.3528]])  tensor([[ -5.2811, -24.8777,   1.2732,   3.2276,  -1.1894,  -3.3144,
           1.9645,   3.7445,   2.2350, -19.3046],
        [  0.5722,   2.6956,  -0.1380,  -0.3497,   0.1289,   0.3591,
          -0.2129,  -0.4057,  -0.2422,   2.0917],
        [ -7.2092, -33.9605,   1.7380,   4.4060,  -1.6236,  -4.5245,
           2.6817,   5.1116,   3.0

## Gradient buffers: `.backward()` and `retain_graph=True`

1. Calling `.backward()` clears the current computation graph.
2. Once `.backward()` is called, intermediate variables used in the construction of the graph are removed.
2. This is used implicitly to let PyTorch know when a new graph is to be built for a new minibatch. This is built around the forward and backward pass paradigm.
3. To retain the graph after the backward pass use `loss.backward(retain_graph=True)`. This lets you re-use intermediate variables to potentially compute a secondary loss after the initial gradients are computed. This is useful to implement things like the gradient penalty in WGANs (https://arxiv.org/abs/1704.00028)

In [18]:
o = torch.mm(x, w) + b
o.backward(torch.ones(5, 10))

### Call backward again -> <font color='red'>This fails</font>

In [21]:
#o = o ** 3
#o.backward(torch.ones(5, 10))

###  But with `retain_graph=True`

In [22]:
o = torch.mm(x, w) + b
o.backward(torch.ones(5, 10), retain_graph=True)

In [23]:
o = o ** 3
o.backward(torch.ones(5, 10))

## <font color='red'>WARNING:</font> Calling `.backward()` multiple times will accumulate gradients into `.grad` and NOT overwrite them.

In [24]:
x = torch.Tensor(5, 3).uniform_(-1, 1).requires_grad_()
w = torch.Tensor(3, 10).uniform_(-1, 1).requires_grad_()
b = torch.Tensor(10,).uniform_(-1, 1).requires_grad_()
print(b.grad)
o = torch.mm(x, w) + b
o.backward(torch.ones(5, 10), retain_graph=True)
print(b.grad)
o.backward(torch.ones(5, 10), retain_graph=True)
print(b.grad)
o.backward(torch.ones(5, 10), retain_graph=True)
print(b.grad)

None
tensor([ 5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.])
tensor([ 10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.])
tensor([ 15.,  15.,  15.,  15.,  15.,  15.,  15.,  15.,  15.,  15.])


## Excluding subgraphs from backward: requires_grad=False, volatile=True & .detach

### `requires_grad=False`

1. If there’s a single input to an operation that requires gradient, its output will also require gradient.

2. Conversely, if all inputs don’t require gradient, the output won’t require it.

3. Backward computation is never performed in the subgraphs where all Variables didn’t require gradients.

4. This is potentially useful when you have part of a network that is pretrained and not fine-tuned, for example word embeddings or a pretrained imagenet model.

In [26]:
x = torch.Tensor(3, 5).uniform_(-1, 1).requires_grad_(False)
y = torch.Tensor(3, 5).uniform_(-1, 1).requires_grad_(False)
z = torch.Tensor(3, 5).uniform_(-1, 1).requires_grad_(True)

In [27]:
o = x + y
print(' o = x + y requires grad ? : %s ' % (o.requires_grad))
o = x + y + z
print(' o = x + y + z requires grad ? : %s ' % (o.requires_grad))

 o = x + y requires grad ? : False 
 o = x + y + z requires grad ? : True 


### `.detach()`

1. It is possible to detach variables from the graph by calling `.detach()`. 
2. This could lead to disconnected graphs. In which case PyTorch will only backpropagate gradients until the point of disconnection.

In [28]:
x = torch.Tensor(3, 5).uniform_(-1, 1).requires_grad_()
y = torch.Tensor(3, 5).uniform_(-1, 1).requires_grad_()
z = torch.Tensor(3, 5).uniform_(-1, 1).requires_grad_()

![caption](images/detach.png)

In [29]:
m1 = x + y
m2 = z ** 2
m1 = m1.detach()
m3 = m1 + m2
m3.backward(torch.ones(3, 5))

In [30]:
print('dm3/dx \n\n %s ' % (x.grad))
print('\ndm3/dy \n\n %s ' % (y.grad))
print('\ndm3/dz \n\n %s ' % (z.grad))

dm3/dx 

 None 

dm3/dy 

 None 

dm3/dz 

 tensor([[-0.2979,  1.7614,  1.4448, -1.9420,  1.2551],
        [-1.8383, -0.5612,  1.8725,  1.1714, -0.4339],
        [ 1.4982,  0.1833, -0.7497,  1.2324, -0.2810]]) 


## Gradients w.r.t intermediate variables in the graph

1. By default, all PyTorch gradient computations w.r.t intermediate nodes in the graph are ad-hoc.

2. This is in the interest of saving memory.

3. To compute gradients w.r.t intermediate variables, use `.retain_grad()` or explicitly compute gradients using `torch.autograd.grad`

4. `.retain_grad()` populates the `.grad` attribute of the Variable while `torch.autograd.grad` returns a Variable that contains the gradients.

In [32]:
x = torch.Tensor(3, 5).uniform_(-1, 1).requires_grad_()
y = torch.Tensor(3, 5).uniform_(-1, 1).requires_grad_()
z = torch.Tensor(3, 5).uniform_(-1, 1).requires_grad_()

In [37]:
m1 = x + y
m2 = z ** 2
#m1.retain_grad()
#m2.retain_grad()
m3 = m1 * m2
m3.backward(torch.ones(3, 5))

In [38]:
print('dm3/dm1 \n\n %s ' % (m1.grad))
print('dm3/dm2 \n\n %s ' % (m2.grad))

dm3/dm1 

 None 
dm3/dm2 

 None 


### In place operations with autograd

source: http://pytorch.org/docs/master/notes/autograd.html

In place operations are suffixed by `_` ex: `log_`, `uniform_` etc.

1. Supporting in-place operations in autograd is difficult and PyTorch discourages their use in most cases.

2. Autograd’s aggressive buffer freeing and reuse makes it very efficient and there are very few occasions when in-place operations actually lower memory usage by any significant amount. Unless you’re operating under heavy memory pressure, you might never need to use them.

### There are two main reasons that limit the applicability of in-place operations:

(a) In-place operations can potentially overwrite values required to compute gradients. Gradient formula of some in-place operations requires the original input (such as `log_`), and while it is possible to recreate it by computing the inverse operation, it is numerically unstable, and requires additional work that often defeats the purpose of using these functions.

(b) Every in-place operation actually requires the implementation to rewrite the computational graph. Out-of-place versions simply allocate new objects and keep references to the old graph, while in-place operations, require changing the creator of all inputs to the Function representing this operation. This can be tricky, especially if there are many Variables that reference the same storage (e.g. created by indexing or transposing), and in-place functions will actually raise an error if the storage of modified inputs is referenced by any other Variable.
In-place correctness checks

## Second and higher order derivatives

### Computing gradients w.r.t gradients

1. `o = xy + z`
2. `l = o + do_dz`

### Practical application of this is gradient penalty (such as WGAN-GP)

In [39]:
x = torch.Tensor(5, 3).uniform_(-1, 1).requires_grad_()
y = torch.Tensor(3, 5).uniform_(-1, 1).requires_grad_()
z = torch.Tensor(5, 5).uniform_(-1, 1).requires_grad_()

In [43]:
o = torch.mm(x, y) + z ** 2
# retain_graph: If False, the graph used to compute the grad will be freed. 
# create_graph: If True, graph of the derivative will be constructed, allowing to compute higher order derivative products.
do_dz = torch.autograd.grad(o, z, grad_outputs=torch.ones(5, 5), retain_graph=True, create_graph=True)
print('do/dz \n\n : %s ' % (do_dz[0]))
l = o ** 3 + do_dz[0]
dl_dz = torch.autograd.grad(l, z, grad_outputs=torch.ones(5, 5))
print('dl/dz \n\n : %s ' % (dl_dz[0]))

do/dz 

 : tensor([[-0.5052, -0.1906, -0.1977,  1.5062,  1.5961],
        [-1.7033, -1.3374, -0.6715,  1.9869, -0.6065],
        [-1.9055,  1.8185, -0.3871,  1.3685,  1.9093],
        [-0.5419,  1.5236, -0.4886,  0.2460,  1.1777],
        [-0.8576,  1.5087, -1.4076, -1.1376,  0.0725]]) 
dl/dz 

 : tensor([[  1.6679,   1.6361,   0.5472,   2.3870,   3.0041],
        [-13.1763,  -4.1508,  -0.0261,   7.2478,   1.3199],
        [  1.8380,   2.0726,   1.3873,   4.2139,   4.8411],
        [ -0.0294,   2.0105,   1.9927,   2.2506,   3.7130],
        [  1.9935,   4.9101,   0.6610,   1.9622,   2.0351]]) 
